<a href="https://colab.research.google.com/github/adarshm11/fake-news-identifier/blob/main/fake_news_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Identifier


Fake news runs rampant in today's society, and in the age of artificial intelligence, it can be difficult to separate what is real from what is false. However, we can use the power of data and machine learning to assess whether news is authentic or not, which is what this project attempts to accomplish.

## Setup

### Step 1: Setup Google Colab and Sync with GitHub

In [8]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install git
!git clone https://github.com/adarshm11/fake-news-identifier.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path 'fake-news-identifier' already exists and is not an empty directory.


### Step 2: Setup Kaggle Connection

In [1]:
!pip install kaggle -q kagglehub
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                         title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
bhadramohit/customer-shopping-latest-trends-dataset         Customer Shopping (Latest Trends) Dataset         76KB  2024-11-23 15:26:12          19177        377  1.0              
hopesb/student-depression-dataset                           Student Depression Dataset.                      454KB  2024-11-22 17:56:03          15685        222  1.0              
oktayrdeki/houses-in-london                                 Houses in London                                  21KB  2024-12-15 19:27:42           1420         27  1.0              
mhassansaboor/intel-stock-data-1980-2024                    I

### Step 3: Import the Kaggle dataset

In [4]:
!python -m pip install --upgrade kagglehub
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download('saurabhshahane/fake-news-classification')

print(os.listdir(path))
csv_file = os.path.join(path, 'WELFake_Dataset.csv')
df = pd.read_csv(csv_file)
print(df.head())

['WELFake_Dataset.csv']
   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  


We can read from the dataset, so we know the connection is successful! Now we can download the dataset to our file menu.

### Step 4: Download the Dataset

In [11]:
!kaggle datasets download -d saurabhshahane/fake-news-classification
import zipfile
from google.colab import drive

with zipfile.ZipFile('fake-news-classification.zip', 'r') as zip_file:
  zip_file.extractall('data')

!ls data

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
License(s): Attribution 4.0 International (CC BY 4.0)
fake-news-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
WELFake_Dataset.csv


### Step 5: Save to Drive

In [12]:
!cp -r data /content/drive/MyDrive/

## Data Preprocessing